# Libraries

In [154]:
import pandas as pd
import seaborn as sns
import regex as re

import spacy

import tqdm

import numpy as np

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import pyLDAvis
import pyLDAvis.gensim_models as gensim_models
import pyLDAvis.gensim_models as gensimvis


import gensim
# from gensim.models import wrappers
# from gensim.models.wrappers import LdaMallet
import gensim.corpora as corpora
from gensim.corpora import dictionary
from gensim.utils import simple_preprocess
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD


import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer


from pprint import pprint

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning)

# Importing -- Fixing columns

In [10]:
full = pd.read_csv('Data/Data-Cleaned/238k-Uncleaned')

In [11]:
# Renaming tweets column, dropping unnamed column, making tweets strings

full['tweets'] = full['0']
full = full.drop(columns = '0')
full.set_index('Unnamed: 0')
# full['tweets'] = full['tweets'].astype('string')
# full.dropna(inplace = True)

,tweets
Unnamed: 0,
1527025892458999814,@testcranker @ITGuy1959 My solar is 100% off g...
1527025892031086593,Boycott #Tesla #TeslaStock #ElonsProblems Vote...
1527025884690993153,@dinodlz @cmclymer Tesla never would have surv...
1527025878974271489,@MarketRebels @elwalvador It would be funny if...
1527025852008976384,@RiceAndrew Right - it's just his nonsense pos...
...,...
1525643350828777473,"@ValueAnalyst1 Yes, beg - borrow and steal to ..."
1525642529567571969,@iamDCinvestor Owh Have to rethink about my th...
1525639115051515909,"@5050isaloss @cardano_whale Overused example, ..."


# Code for Sentiment ---- ---- NOT NEEDED FOR MODELING

In [8]:
# CODE WORKS --- JUST USE IT LATER 

# Adding Sentiment
sia = SentimentIntensityAnalyzer()
comp_dic = {}
for i, tweet in full['tweets'].iteritems():
    comp_dic[i] = sia.polarity_scores(tweet)['compound']

comp_scores = pd.Series(comp_dic, name = 'sentiment')
df_sent = full.merge(comp_scores, left_index = True, right_index = True)
df = df_sent.set_index('Unnamed: 0')

# Preprocessing ---- will need to test different preprocessing later

In [ ]:
"""
The goal here is to get to a final model with the full data and an iteration of cleaned preprocessing
Clean this notebook as much as possible
make a copy of the notebook-- save the original and MUST get bi and trigrams to work


"""

In [12]:
def Series_Preprocessor(tweet):
    stopWords = stopwords.words('english')
    stopWords.extend(['tesla', 'c,', 'x', 't', 'p', 'amp'])
    ps = PorterStemmer()
    wnl = WordNetLemmatizer()

        # helper function to change nltk's part of speech tagging to a wordnet format.
    def pos_tagger(nltk_tag):
        if nltk_tag.startswith('J'):
            return wordnet.ADJ
        elif nltk_tag.startswith('V'):
            return wordnet.VERB
        elif nltk_tag.startswith('N'):
            return wordnet.NOUN
        elif nltk_tag.startswith('R'):
            return wordnet.ADV
        else:         
            return None 
    
#   Lowercase all 
    tweet = tweet.lower()
    
#   Removing @ handles, links-- strip whitespace breaks and tabs
    tweet = re.sub(r"@\w+|http\S+", "", tweet).strip().replace("\r", "").replace("\n", "").replace("\t", "")
    
#   Tokenize,make list of words, removing punctuation and stopwords
    tweet = [x for x in word_tokenize(tweet) if ((x.isalpha()) & (x not in stopWords)) ]
    
#   Map part of speech tags to words and use words/tags to lemmatize accuratley
    tweet = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tag(tweet))) 
    tweet = " ".join([wnl.lemmatize(x[0], x[1]) for x in tweet if x[1] is not None])

    return tweet 

In [13]:
df = full['tweets'].apply(Series_Preprocessor)

## Need to make Bi and Tri Grams ---- Do this below

In [14]:
# remove punctuation
full['preprep'] = full['tweets'].map(lambda x: re.sub('[,\.!?]', '', x))
# Lowercase
full['preprep'] = full['preprep'].map(lambda x: x.lower())
# #   Removing @ handles, links-- strip whitespace breaks and tabs
full['preprep'] = full['preprep'].map(lambda x: re.sub(r"@\w+|http\S+", "", x).strip().replace("\r", "").replace("\n", "").replace("\t", ""))

In [15]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = full.preprep.values.tolist()
data_words = list(sent_to_words(data))

In [16]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=3, threshold=75) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=60)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [17]:
stop_words = stopwords.words('english')
stop_words.extend(['tesla', 'c,', 'x', 't', 'p', 'amp', 'car', 'go'])
wnl = WordNetLemmatizer()

In [18]:
# Functions to for preprocessing for bigrams and trigrams texts, all tested and edited all should be good 

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words]for doc in texts]

def make_bigrams(texts):
    return bigram_mod[texts]

def make_trigrams(texts):
    return trigram_mod[bigram_mod[texts]]

def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None 
    
def lemmatize_texts(tweet):
     for i in tweet:
        word = (map(lambda x: (x[0], pos_tagger(x[1])), pos_tag(tweet))) 
        word_lem = " ".join([wnl.lemmatize(x[0], x[1]) for x in word if x[1] is not None])
        return word_lem

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# lemmatize
data_lemmatized = [lemmatize_texts(tweet) for tweet in data_words_bigrams]

In [344]:
a = [(i) for i,x  in enumerate(data_lemmatized) if x ==None]
len(a)

659

In [325]:
#proved that Nones are introduced during the lemmatization step -- there are no nones before this step
# This will need to be fixed later
lem_noNone = [x for x in data_lemmatized if x is not None]
print(len(lem_noNone))

237000


## Find missing values after making min tweet length 4 words maybe =============================

In [300]:
# testing enumerate with above lemmatization function
tester = [(i,x) for (i,x) in enumerate(data_lemmatized) if x is not None]

def find_missing(lst):
    return [x for x in range(lst[0], lst[-1]+1) 
                               if x not in lst] 
lst = [tup[0] for tup in tester]
ind = (find_missing(lst))
# ind

In [23]:
lem_noNone = [i.split() for i in lem_noNone]

In [24]:
id2word = corpora.Dictionary(lem_noNone)
print(len(id2word)) 

66590


In [25]:
# This can be Tuned --- and probably chould be

id2word.filter_extremes(no_below=6, no_above=.90)
print(len(id2word))

18507


In [301]:
# Another potential preprocessing step 
# Could remove tweets that are fewer than n number of words 

counter = 0
for i in lem_noNone:
    if len(i) <= 3:
        counter += 1
counter

32702

In [26]:
corpus = [id2word.doc2bow(d) for d in lem_noNone]

In [42]:
# Instantiating a Base LDA model
base_model = LdaMulticore(corpus=corpus,
                          num_topics=5,
                          id2word=id2word,
                          workers=7,
                          passes=5,
                          chunksize=2500,
                          random_state = 42)
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]
topics = [' '.join(t[0:10]) for t in words]

for id, t in enumerate(topics): 
    print(f"------ Topic {id + 1} ------")
    print(t, end="\n\n")

------ Topic 1 ------
model first spot time think teslaradar day say know standard

------ Topic 2 ------
buy musk elon car people electric make want think say

------ Topic 3 ------
car electric make ev company battery year get vehicle market

------ Topic 4 ------
stock buy get go twitter price share market see back

------ Topic 5 ------
fire door car people driver crash drive use get autopilot



In [43]:
# a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=lem_noNone, 
                                   dictionary=id2word, coherence='c_v')

coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base, '\n\nPerplexity: ', base_perplexity)


Coherence Score:  0.33998669503645845 

Perplexity:  -7.642658044311896


In [40]:
"""
chunk size = 4000
Coherence Score:  0.3277361656874404 
Perplexity:  -7.656176410369252



chunk size = 1000
Coherence Score:  0.36408864500672655 
Perplexity:  -7.648354363180408"""

'\nchunk size = 1000\nCoherence Score:  0.36408864500672655 \nPerplexity:  -7.648354363180408'

In [41]:
pyLDAvis.enable_notebook()
gensimvis.prepare(base_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.043024  0.031502       1        1  29.594909
1      0.088135 -0.110200       2        1  23.999476
3      0.032734 -0.103158       3        1  19.352818
4      0.069061  0.179123       4        1  17.264735
0     -0.232954  0.002733       5        1   9.788061, topic_info=        Term          Freq         Total Category  logprob  loglift
162    model  11888.000000  11888.000000  Default  30.0000  30.0000
156     elon  17298.000000  17298.000000  Default  29.0000  29.0000
151    stock  24140.000000  24140.000000  Default  28.0000  28.0000
128     musk  19014.000000  19014.000000  Default  27.0000  27.0000
144      buy  26776.000000  26776.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
127   market   1233.367149   9215.511780   Topic5  -5.2639   0.3129
421  someone    908.371327   3200.536320   Topic5  -5.5697   1.0646
151    stock   1258.042049  24140.654290   Topic5  -5.2441  -0.6303
306     good    944.697435  10847.150277   Topic5  -5.5305  -0.1168
94       new    913.154553   8170.111762   Topic5  -5.5645   0.1327

[472 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1609      1  0.011131  accident
1609      2  0.004281  accident
1609      3  0.007706  accident
1609      4  0.974361  accident
1609      5  0.002569  accident
...     ...       ...       ...
355       1  0.507355      year
355       2  0.128376      year
355       3  0.158348      year
355       4  0.045131      year
355       5  0.160860      year

[1289 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 4, 5, 1])

#### ============================================================================

# Tuning topics and pther things .....

### Here
## We 
# Go

In [63]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=1500,
                                           passes=8,
                                           alpha=a,
                                           eta=b, 
                                           workers = 7)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=lem_noNone, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [76]:
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 3
max_topics = 9
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [#gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.5)), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['50% Corpus', '75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    results_df = pd.DataFrame(model_results)
    pbar.close()


  1%|█▍                                                                                                                                                          | 3/336 [31:03:59<3448:23:05, 37279.84s/it]

 23%|████████████████████████████████████▋                                                                                                                           | 77/336 [3:05:37<20:17:40, 282.09s/it]


 46%|█████████████████████████████████████████████████████████████████████████▎                                                                                     | 155/336 [8:39:49<12:53:32, 256.42s/it]


 69%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                 | 233/336 [12:45:07<2:08:49, 75.04s/it]


 93%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉            | 311/336 [14:28:07<32:46, 78.68s/it]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 336/336 [14:58:14<00:00, 82.49s/it]
337it [14:59:40, 83.50s/it]                                                                                                                                                                                 
338it [15:01:08, 84.74s/it]
339it [15:02:33, 85.03s/it]
340it [15:03:56, 84.44s/it]
341it [15:05:04, 79.33s/it]
342it [15:06:41, 84.86s/it]
343it [15:08:26, 90.72s/it]
344it [15:10:10, 94.72s/it]
345it [15:11:36, 92.26s/it]
346it [15:12:52, 87.39s/it]
347it [15:14:05, 82.84s/it]
348it [15:15:22, 81.12s/it]
349it [15:16:46, 81.93s/it]
350it [15:17:59, 79.41s/it]
351it [15:19:21, 80.06s/it]
352it [15:20:41, 80.09s/it]
353it [15:22:04, 80.87s/it]
354it [15:23:33, 83.39s/it]
355it [15:24:53, 82.34s/it]
356it [15:26:12, 81.51s/it]
357it [15:27:36, 82.02s/it]
358it [15:29:01, 82.97s/it]
35

In [5]:
results_df = pd.read_csv('Data/first-gs-results-24h')
results_df.drop(columns = 'Unnamed: 0', inplace = True)

In [ ]:
results_df[(results_df['Coherence'] >= 0.38) & (results_df['Topics'] == 5 )]

# =============================================================================

# Below, will be trying to reduce size of corpus to increase model coherence

In [94]:
len(lem_noNone)

237000

In [304]:
# Removing Tweets fewer than N number of words 
counter = []
for i, text in enumerate(lem_noNone):
    if len(text) <= 6:
        counter.append(i)
len(counter)

81052

In [96]:
longer_tweets = [(tweet) for (i, tweet) in enumerate(lem_noNone) if i not in counter]
len(longer_tweets)

In [166]:
new_id2word = corpora.Dictionary(longer_tweets)
print(len(new_id2word)) 

62118


In [167]:
new_id2word.filter_extremes(no_below=9, no_above=.90)
print(len(new_id2word))

12165


In [168]:
new_corpus = [new_id2word.doc2bow(d) for d in longer_tweets]

In [170]:
# Instantiating a Base LDA model
newer_model = LdaMulticore(corpus=new_corpus,
                          num_topics=5,
                          id2word=new_id2word,
                          workers=6,
                          passes=8,
                          alpha = 0.91 , 
                          eta = 0.61,
                          chunksize=1500,
                          random_state = 100)
newer_words = [re.findall(r'"([^"]*)"',t[1]) for t in newer_model.print_topics()]
newer_topics = [' '.join(t[0:10]) for t in newer_words]

for id, t in enumerate(newer_topics): 
    print(f"------ Topic {id + 1} ------")
    print(t, end="\n\n")

------ Topic 1 ------
model fire first time crash spot someone news autopilot show

------ Topic 2 ------
car get electric drive door use know people vehicle need

------ Topic 3 ------
buy stock musk elon twitter get want people price money

------ Topic 4 ------
make car electric battery world company day work build solar

------ Topic 5 ------
ev market company year make much see go good tsla



In [172]:
newer_perplexity = newer_model.log_perplexity(new_corpus)

# Compute Coherence Score
newer_coherence_model = CoherenceModel(model= newer_model, texts=longer_tweets, 
                                   dictionary=new_id2word, coherence='c_v')

new_coherence_lda_model_base = newer_coherence_model.get_coherence()
print('\nCoherence Score: ', new_coherence_lda_model_base, '\nPerplexity: ', newer_perplexity)


Coherence Score:  0.35341664092529373 
Perplexity:  -7.507813170245735


In [173]:
pyLDAvis.enable_notebook()
gensimvis.prepare(newer_model, new_corpus, new_id2word)

/Users/austinboring/opt/anaconda3/envs/broke-the-other/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/austinboring/opt/anaconda3/envs/broke-the-other/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/austinboring/opt/anaconda3/envs/broke-the-other/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/austinboring/opt/anaconda3/envs/broke-the-other/lib/python3.10/site-packages/past/builtins/misc.py:

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.083917  0.238382       1        1  22.851046
1     -0.036160 -0.083506       2        1  21.753162
4     -0.121787  0.007618       3        1  21.448634
3     -0.070020 -0.179605       4        1  18.901655
0      0.311883  0.017111       5        1  15.045503, topic_info=         Term          Freq         Total Category  logprob  loglift
129     stock  19025.000000  19025.000000  Default  30.0000  30.0000
122       buy  20103.000000  20103.000000  Default  29.0000  29.0000
140     model  11823.000000  11823.000000  Default  28.0000  28.0000
97        car  33295.000000  33295.000000  Default  27.0000  27.0000
108      musk  15769.000000  15769.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
1165  include   1277.662333   1572.518033   Topic5  -5.5077   1.6864
220       say   2395.888031  11599.333692   Topic5  -4.8789   0.3169
291   vehicle   1583.723253   7119.373480   Topic5  -5.2929   0.3910
999      case   1279.661122   1688.516940   Topic5  -5.5061   1.6168
151      look   1491.937383   7088.398109   Topic5  -5.3526   0.3357

[374 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
428       1  0.044921  agree
428       2  0.008858  agree
428       3  0.925621  agree
428       4  0.018348  agree
428       5  0.001898  agree
...     ...       ...    ...
295       1  0.012901   year
295       2  0.325768   year
295       3  0.605053   year
295       4  0.041952   year
295       5  0.014279   year

[1585 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 5, 4, 1])

# =============================================================================

# Below --- Need to link tweets to topics, and calculate sentiment by topic

In [178]:
full

,Unnamed: 0,tweets,preprep
0,1527025892458999814,@testcranker @ITGuy1959 My solar is 100% off g...,my solar is 100% off grid it runs the computer...
1,1527025892031086593,Boycott #Tesla #TeslaStock #ElonsProblems Vote...,boycott #tesla #teslastock #elonsproblems vote...
2,1527025884690993153,@dinodlz @cmclymer Tesla never would have surv...,tesla never would have survived without the ca...
3,1527025878974271489,@MarketRebels @elwalvador It would be funny if...,it would be funny if his fanbase just took pro...
4,1527025852008976384,@RiceAndrew Right - it's just his nonsense pos...,right - it's just his nonsense positions i do ...
...,...,...,...
237654,1525643350828777473,"@ValueAnalyst1 Yes, beg - borrow and steal to ...",yes beg - borrow and steal to invest in $tsla#...
237655,1525642529567571969,@iamDCinvestor Owh Have to rethink about my th...,owh have to rethink about my thesis i always t...
237656,1525639115051515909,"@5050isaloss @cardano_whale Overused example, ...",overused example but elon musk was alienated f...
237657,1525633771009101828,@DomRobins0n @JoJoFromJerz Founded spaceX a ve...,founded spacex a very successful and also argu...


In [175]:
len(new_corpus)

155948

In [192]:
# Counter contains perfect indices of all tweets that are short -- directly translates to original full dataframe
counter[:10]

[1, 3, 7, 9, 10, 12, 15, 20, 24, 25]

In [191]:
lem_noNone[19]

['elon',
 'mad',
 'look',
 'stock',
 'past',
 'month',
 'actual',
 'stock',
 'chart',
 'past',
 'month',
 'poorelon']

In [183]:
fuck = [1, 24, 3]
for i in enumerate(fuck):
    print(i)

(0, 1)
(1, 24)
(2, 3)


In [ ]:
"""
Note above, 
the original dataframe is indexed from 0, as in it is in the same form as an enumerated list
we created a function earlier to find all of the missing index values somewhere around creating lem_noNone

this function can be used to set proper indexs on the new lists past the funnkiness somehow... easier said than done 
"""

## Attempting to get the top documents per topic 

In [194]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


In [195]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=newer_model, corpus=new_corpus, texts=longer_tweets)
df_topic_sents_keywords

,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
0,3,0.5235,"make, car, electric, battery, world, company, ...","[solar, grid, run, computer, radio, fridge, en..."
1,2,0.3959,"buy, stock, musk, elon, twitter, get, want, pe...","[never, survive, carbon_offsets, implement, de..."
2,2,0.5352,"buy, stock, musk, elon, twitter, get, want, pe...","[right, nonsense, position, find, sort, hard, ..."
3,4,0.6391,"ev, market, company, year, make, much, see, go...","[ford, pe, ration, pe_ratio, tsla, still, long..."
4,1,0.5407,"car, get, electric, drive, door, use, know, pe...","[think, back, time, work, employee, brag, work..."
...,...,...,...,...
155943,4,0.4560,"ev, market, company, year, make, much, see, go...","[yes, beg, borrow, steal, invest, tsla, world,..."
155944,3,0.3583,"make, car, electric, battery, world, company, ...","[owh, rethink, thesis, always, think, regulati..."
155945,2,0.3730,"buy, stock, musk, elon, twitter, get, want, pe...","[overused, example, elon, musk, alienate, almo..."
155946,3,0.3681,"make, car, electric, battery, world, company, ...","[found, spacex, successful, also, arguably, in..."


In [196]:
sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

,Topic_Num,Topic_Perc_Contrib,Keywords,Representative Text
0,0,0.8515,"model, fire, first, time, crash, spot, someone...","[scottsdale_community, college, wal_mart, marr..."
1,1,0.8622,"car, get, electric, drive, door, use, know, pe...","[wow, wrong, drive, old, shitty, vehicle, ii, ..."
2,2,0.8577,"buy, stock, musk, elon, twitter, get, want, pe...","[ceo, elon, musk, bid, purchase, twitter, stri..."
3,3,0.8408,"make, car, electric, battery, world, company, ...","[blockstream, block, team, solar, power, btc, ..."
4,4,0.8488,"ev, market, company, year, make, much, see, go...","[ev, unit, sale, rise, increased, sale, volume..."


In [201]:
sent_topics_sorteddf_mallet.iloc[2]['Keywords']

'buy, stock, musk, elon, twitter, get, want, people, price, money'

In [203]:
# sent_topics_sorteddf_mallet.iloc[2]['Representative Text']

In [273]:
# longer_tweets[:10]

In [258]:
df_topic_sents_keywords

,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
0,3,0.5235,"make, car, electric, battery, world, company, ...","[solar, grid, run, computer, radio, fridge, en..."
1,2,0.3959,"buy, stock, musk, elon, twitter, get, want, pe...","[never, survive, carbon_offsets, implement, de..."
2,2,0.5352,"buy, stock, musk, elon, twitter, get, want, pe...","[right, nonsense, position, find, sort, hard, ..."
3,4,0.6391,"ev, market, company, year, make, much, see, go...","[ford, pe, ration, pe_ratio, tsla, still, long..."
4,1,0.5407,"car, get, electric, drive, door, use, know, pe...","[think, back, time, work, employee, brag, work..."
...,...,...,...,...
155943,4,0.4560,"ev, market, company, year, make, much, see, go...","[yes, beg, borrow, steal, invest, tsla, world,..."
155944,3,0.3583,"make, car, electric, battery, world, company, ...","[owh, rethink, thesis, always, think, regulati..."
155945,2,0.3730,"buy, stock, musk, elon, twitter, get, want, pe...","[overused, example, elon, musk, alienate, almo..."
155946,3,0.3681,"make, car, electric, battery, world, company, ...","[found, spacex, successful, also, arguably, in..."


In [215]:
df_topic_sents_keywords[(df_topic_sents_keywords['Dominant_Topic'] == 4) & (df_topic_sents_keywords['Perc_Contribution'] >= 0.84)]

,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
6991,4,0.8404,"ev, market, company, year, make, much, see, go...","[ford, almost, percent, gross_margin, last, qu..."
11628,4,0.8488,"ev, market, company, year, make, much, see, go...","[ev, unit, sale, rise, increased, sale, volume..."
32472,4,0.8404,"ev, market, company, year, make, much, see, go...","[ford, almost, percent, gross_margin, last, qu..."
78510,4,0.8400,"ev, market, company, year, make, much, see, go...","[aapl, also, overvalue, apple, revenue, increa..."
85674,4,0.8410,"ev, market, company, year, make, much, see, go...","[probably, best, fundamental, market, fast, gr..."
98426,4,0.8410,"ev, market, company, year, make, much, see, go...","[probably, best, fundamental, market, fast, gr..."
104646,4,0.8410,"ev, market, company, year, make, much, see, go...","[probably, best, fundamental, market, fast, gr..."
125447,4,0.8409,"ev, market, company, year, make, much, see, go...","[probably, best, fundamental, market, fast, gr..."
142320,4,0.8410,"ev, market, company, year, make, much, see, go...","[probably, best, fundamental, market, fast, gr..."


In [302]:
len(ind)

659

In [305]:
len(counter)

81052

In [306]:
sum_dropped = counter + ind
len(sum_dropped)

81711

In [307]:
# full_tester = full.drop(full.index[ind])
full_tester = full.drop(full.index[sum_dropped])
full_tester.reset_index(inplace = True)

In [311]:
full_tester

,index,Unnamed: 0,tweets,preprep
0,0,1527025892458999814,@testcranker @ITGuy1959 My solar is 100% off g...,my solar is 100% off grid it runs the computer...
1,2,1527025884690993153,@dinodlz @cmclymer Tesla never would have surv...,tesla never would have survived without the ca...
2,4,1527025852008976384,@RiceAndrew Right - it's just his nonsense pos...,right - it's just his nonsense positions i do ...
3,5,1527025846988382208,@H0NIS0IT @patricklarkin @SRuhle Ford PE ratio...,ford pe ration is 10 tesla pe ratio is 100 tsl...
4,6,1527025833809809408,Thinking back to that time I worked at Tesla a...,thinking back to that time i worked at tesla a...
...,...,...,...,...
156195,237654,1525643350828777473,"@ValueAnalyst1 Yes, beg - borrow and steal to ...",yes beg - borrow and steal to invest in $tsla#...
156196,237655,1525642529567571969,@iamDCinvestor Owh Have to rethink about my th...,owh have to rethink about my thesis i always t...
156197,237656,1525639115051515909,"@5050isaloss @cardano_whale Overused example, ...",overused example but elon musk was alienated f...
156198,237657,1525633771009101828,@DomRobins0n @JoJoFromJerz Founded spaceX a ve...,founded spacex a very successful and also argu...


In [277]:
full_tester.tweets.iloc[104646]

'just extremely bad memories of the one single time I was in a Tesla'

In [312]:
len(longer_tweets)

155948

In [270]:
len(new_corpus)

155948

In [354]:
def find_common_strings(string_list, input_string):
    common_strings = []
    for s in string_list:
        if set(s).intersection(set(input_string)) == set(s):
            common_strings.append(s)
    return common_strings

In [397]:
def find_common_strings(string_list, input_string):
    common_strings = []
    for i, s in enumerate(string_list):
        if set(s).intersection(set(input_string)) == set(s):
            common_strings.append((i, s))
            
    pos_dic = {}
    for i, string in common_strings:
        string_split = string.split(' ')
        input_split = input_string.split(' ')
        common = []
        for word in input_split:
            if word in string_split:
                common.append(word)
        pos_dic[i] = common
        
    
    
    
    return common_strings, pos_dic

In [361]:
test_string = (' ').join(df_topic_sents_keywords[0].iloc[0])
test_string

'solar grid run computer radio fridge entire yard big enough recharge'

In [398]:
big_list, ind_dict = find_common_strings(full['preprep'], test_string)

In [399]:
print(len(ind_dict), len(big_list))

7460 7460


In [400]:
potentials = []
for key, value in ind_dict.items():
    if value != []:
        potentials.append(key)

In [401]:
for i in potentials:
    print(full['preprep'].iloc[i])

i guess tesla had a good mediocre run
oil is the big money for many months to come
hey  any plan about solar panels in belgium europe
this could be big for tesla
one day imma post me my tesla my big ole house her left thigh and log out
see if you can get some discounts on a solar array
so yeah a tesla running off fossil fuels at the grid is still better than a petrol car
other than it is correct tesla is a terrible company run by terrible people
big time to the point that i am considering a ford mustang instead of a tesla
elon big mad
tesla the republican run car company
though tesla has not fallen enough yet
fair enough
run ruby and tesla laughing
cars need a solar cell on roof
put solar panels on all tesla cars during day to help charge batteries in them
and the irony that tesla cars run on dc rather than ac is not lost
then you better run out and buy a tesla stat
sigh i already did tesla solar roof tiles and a house battery
first heat pumps and an electric thermal storage unit then 

In [352]:
test_string[::-1].find(' ')

8

In [347]:
for s in full['preprep']:
    

In [353]:
# string_test = find_common_strings(lem_noNone,test_string )
# string_test

# ========================================================================================================================================================================================================================================================================================================================================================================

# SCRAP

# ========================================================================================================================================================================================================================================================================================================================================================================

In [ ]:
for i in range(len(doc_set)):
    print(ldamodel[corpus[i]])

In [ ]:
vectorizer = CountVectorizer()
data_vectorized = vectorizer.fit_transform(df.drop('tokens'))



# CV Param
search_params = {'n_components': [5, 10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9], 'batch_size':[128, 256]}

# Init Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, cv = 5, param_grid=search_params, n_jobs = -1)



# Grid Search
model.fit(data_vectorized)

# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))


best_lda_model

In [ ]:
GridSearchCV(cv=None, error_score='raise',
             estimator=LatentDirichletAllocation(batch_size=128, 
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1, 
                                                 learning_decay=0.7, 
                                                 learning_method=None,
                                                 learning_offset=10.0, 
                                                 max_doc_update_iter=100, 
                                                 max_iter=10,
                                                 mean_change_tol=0.001, 
                                                 n_components=10, 
                                                 n_jobs=1,
                                                 perp_tol=0.1, 
                                                 random_state=None,
                                                 topic_word_prior=None, 
                                                 total_samples=1000000.0, 
                                                 verbose=0),
             n_jobs=-2,
             param_grid={'n_topics': [10, 15, 20, 30], 
                         'learning_decay': [0.5, 0.7, 0.9]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
             scoring=None, verbose=0)